In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Custom AlphaDropout Layer Implementation
class CustomAlphaDropout(layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomAlphaDropout, self).__init__(**kwargs)
        self.rate = rate
        self.alpha = -1.7580993408473766  # Fixed-point value for SELU
        self.scale = 1.0507009873554805   # Scale factor for SELU

    def call(self, inputs, training=None):
        if training:
            # Create a dropout mask
            mask = tf.random.uniform(shape=tf.shape(inputs)) >= self.rate
            mask = tf.cast(mask, inputs.dtype)

            # Apply the mask and adjust the dropped values
            dropped_inputs = mask * inputs + (1 - mask) * self.alpha
            dropped_inputs *= 1.0 / tf.sqrt(self.rate * self.alpha**2 + (1 - self.rate))

            return dropped_inputs
        else:
            return inputs

# Example model using CustomAlphaDropout
def create_model_with_custom_alpha_dropout(dropout_rate=0.3):
    model = Sequential([
        layers.Input(shape=(2,)),
        layers.Dense(64, activation='selu', kernel_initializer='lecun_normal'),
        CustomAlphaDropout(rate=dropout_rate),
        layers.Dense(64, activation='selu', kernel_initializer='lecun_normal'),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Generating dummy data
X = np.random.randn(1000, 2)
y = np.random.randint(0, 2, size=(1000,))
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Creating and training the model
model = create_model_with_custom_alpha_dropout(dropout_rate=0.3)
history = model.fit(X, y, epochs=10, validation_split=0.2, verbose=1)

# Print model summary
model.summary()


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.4836 - loss: 0.8883 - val_accuracy: 0.4700 - val_loss: 0.7609
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.5232 - loss: 0.7785 - val_accuracy: 0.5450 - val_loss: 0.7148
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4812 - loss: 0.7592 - val_accuracy: 0.5150 - val_loss: 0.7052
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5072 - loss: 0.7223 - val_accuracy: 0.5550 - val_loss: 0.7025
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4909 - loss: 0.7188 - val_accuracy: 0.5050 - val_loss: 0.7051
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4684 - loss: 0.7373 - val_accuracy: 0.5300 - val_loss: 0.7161
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4924 - loss: 0.7269 - val_accuracy: 0.4800 - val_loss: 0.7215
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5325 - loss: 0.7099 - val_accuracy: 0.5200 - va

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                     │ (None, 64)                  │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_alpha_dropout_1               │ (None, 64)                  │               0 │
│ (CustomAlphaDropout)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,448 (52.54 KB)

 Trainable params: 4,482 (17.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,966 (35.03 KB)